In [17]:
from bs4 import BeautifulSoup
import pandas as pd
import os
import requests
import urllib3
from tqdm import tqdm
import time
import re

In [24]:
data=pd.read_excel('input.xlsx')

In [23]:
#function for getting  stopwords 
#stopwords taken from  https://sraf.nd.edu/textual-analysis/resources/ 

def stopwords_list():
    stop_words=set()
    with open('StopWords_Generic.txt','r') as f:
        for line in f:
            stop_words.add(line.strip().lower())
    #print('STOPWORDS ARE ', stop_words)
    return stop_words

In [22]:
#using LoughranMcDonald_MasterDictionary_2020 of words
#dict link https://sraf.nd.edu/textual-analysis/resources/ 
if not os.path.isdir('dicts'):
    os.mkdir('dicts')
def dictinoaries():
    
    stop_words=stopwords_list()
    pos_neg_dict=pd.read_csv('LoughranMcDonald_MasterDictionary_2020.csv')
    pos_neg_dict['Word']=pos_neg_dict['Word'].apply(lambda row:str(row).lower()) #words to lower
    cstm_pos_dict={}
    cstm_neg_dict={}
    with open('dicts//neg_dict.txt','w') as f:
        for word,score in zip(pos_neg_dict['Word'],pos_neg_dict['Negative']):
            if word not in stop_words and score!=0:
                cstm_neg_dict[word]=-1
                f.write(word+'\n')
            
            
    with open('dicts//pos_dict.txt','w') as f:
        for word,score in zip(pos_neg_dict['Word'],pos_neg_dict['Positive']):
            if word not in stop_words and score!=0:
                cstm_pos_dict[word]=+1
                f.write(word+'\n')
    return cstm_neg_dict,cstm_pos_dict

In [ ]:
#function to extract positive words from the text which are in the dict and not in the stop words list
#and calculating positive and negative score
def extraction(text):
    text=text.lower()
    #dictinoary
    cstm_neg_dict,cstm_pos_dict=dictinoaries()
    #stopwords
    stop_words=stopwords_list()
    tokenizer=nltk.RegexpTokenizer('\w+')
    #from nltk.tokenize import word_tokenize   #also includes . ? etc
    token_text=tokenizer.tokenize(text)
    #token_text=word_tokenize(text)
    pos_score_dict={}
    for word in token_text:
         if word not in stop_words and word in cstm_pos_dict:
                print('word is',word)
                if word not in pos_score_dict:
                    pos_score_dict[word]=1
                else:
                    pos_score_dict[word]+=1
                
    
    #negtive_score
    neg_score_dict={}
    for word in token_text:
         if word not in stop_words and word in cstm_neg_dict:
                if word not in neg_score_dict:
                    neg_score_dict[word]=-1
                else:
                    neg_score_dict[word]-=1
                
    
    text_pos_score =0
    text_neg_score=0
    #adding all the scores
    text_pos_score=sum(pos_score_dict.values())
    text_neg_score=sum(neg_score_dict.values())*-1
    
    #POLARITY SCORE
    text_polarity_score=(text_pos_score-text_neg_score)/((text_pos_score + text_neg_score) + 0.000001)
    
    #Subjectivity Score: 
    clean_text=[word for word in token_text if word not in stop_words]
    #print('clean_text ',len(clean_text))
    #a=' '.join(str(word) for word in clean_text) in str spaces are also counted in len
    
    subjectivity_score=(text_pos_score + text_neg_score)/(len(clean_text)+ 0.000001)
    #print('subjectivity_score', subjectivity_score)
    
    return text_pos_score,text_neg_score,text_polarity_score,subjectivity_score

In [20]:
#main function
   
def scrape(iid,link):
    
    headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.12; rv:55.0) Gecko/20100101 Firefox/55.0',
}
    html_txt=requests.get(str(link),headers=headers)
    
    clean_text=BeautifulSoup(html_txt.content,'html.parser')
    heading=BeautifulSoup.findAll(clean_text,'h1',attrs={'class':'entry-title'})
    #print(heading)
    heading=re.sub(r'<.*?>','',str(heading))
    heading=heading.strip("[]")
    #print(heading)
    #paragrapsh
    paragraph=BeautifulSoup.findAll(clean_text,'p')
    #print(paragraph)
    paragraph=re.sub(r'<.*?>','',str(paragraph))
    paragraph=paragraph.strip('[]') #[] i the beg and the ending
    print(paragraph)
    #print(paragraph)
    #with open('scraped_files_dummy\\'+str(iid)+'.txt','w') as f:
        #f.write(heading)
        #f.write(paragraph)
    text=heading+" . "+paragraph
    extraction(text)
    time.sleep(10)
    
    return text,text_pos_score,text_neg_score,text_polarity_score,subjectivity_score
    

In [26]:
text=[]
text_pos_score=[]
text_neg_score=[]
text_polarity_score=[]
subjectivity_score=[]

if not os.path.isdir('scraped_files'):
    os.mkdir('scraped_files')
for url_id,link in tqdm(zip(data['URL_ID'],data['URL'])):
    text_single,text_pos,text_neg,text_polarity,subjectivity=scrape(url_id,link)
    text.append(text_single)
    text_pos_score.append(text_pos)
    text_neg_score.append(text_neg_score)
    text_polarity_score.append(text_polarity)
    subjectivity_score.append(subjectivity)
    

In [28]:
data['text']=text
data['postive']=text_pos_score
data['negative']=text_neg_score
data['polarity']=text_polarity_score
data['subjectivity']=subjectivity_score